# $\S 2$ Interpolation Methods 
---

In this lab, I will test three algorithms of solving no-linear equation about their correctness and efficiency, they are: 
- Lagrange interpolation
- Newtonian interpolation
- cubic spline interpolation   
And then use them to solve 2 problems in instruction book.

## 1. realization of these algorithms 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 

### 1.1. Lagrange Interpolation 

Formualr of Interpolation:   
given that $n+1$ points $(x_i, y_i),\ i\in\{0, 1, 2, ..., n\}$ on function $f(x)$. They follow the order: $x_0<x_1<x_2<...<x_n$. we can interplate it by following function with $n^{th}$ largrange method: 
$$
p_n(x) = \sum_{i=0}^{n} y_i\cdot l_i(x) \\ 
\Rightarrow \sum_{i=0}^{n} y_i \cdot \prod_{j=0, j \ne i}^{n} {x - x_j \over x_i - x_j} \tag{1}
$$

In [ ]:
def lagrange(n, x, y):
    ll = []
    for i in range(n):
        f = lambda t: 1
        for j in range(n): # calcualte l_i
            if j != i:
                f = lambda t: f(t) * (t - x[j]) / (x[i] - x[j]) 
        
        ll.append(f)  # append the l_i
    
    larg = lambda t: 0
    for i in range(n):
        larg = lambda t: larg(t) + ll[i](t) * y[i]
    
    return larg

### 1.2. Newtonian Interpolation 

Formualr of Interpolation:   
given that $n+1$ points $(x_i, y_i),\ i\in\{0, 1, 2, ..., n\}$ on function $f(x)$. They follow the order: $x_0<x_1<x_2<...<x_n$. we can interplate it by following function with $n^{th}$ Newtonian method: 
$$
p_n(x) = f(x_0) + \sum_{i=1}^{n} f[x_0, x_1,..., x_k] \cdot \omega_k(x) \tag{2}
$$

In [ ]:
def newton(n, x, y):
    